<a href="https://colab.research.google.com/github/Gonzy/tests1/blob/master/ResumableLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics --upgrade -q > /dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive/datasets/

Mounted at /content/gdrive
project_1  test.zip


In [ ]:
!mkdir -p /content/merged
!cp /content/gdrive/MyDrive/datasets/project_1/merged.tar.gz /content/merged.tar.gz
!tar -xzvf merged.tar.gz -C /content/merged > /dev/null

In [ ]:
#!cp /content/gdrive/MyDrive/datasets/test.zip /content/test.zip
#!unzip test.zip -d /content/test_dataset > /dev/null

In [ ]:
# !rm -rf /content/gdrive/MyDrive/datasets/checkpoints/project_1/
# !mkdir /content/gdrive/MyDrive/datasets/project_1/checkpoints/

In [ ]:
!mkdir -p /content/runs/detect/scratch-detector
%cd /content/runs/detect/scratch-detector
!tar -xzvf /content/gdrive/MyDrive/datasets/project_1/checkpoints/last.tar.gz > /dev/null

In [1]:
%cd /content/

from ultralytics import YOLO
import os
import shutil
from pathlib import Path

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

checkpoint_dir = "/content/gdrive/MyDrive/datasets/project_1/checkpoints"
last_filename = "last.tar.gz"
last_gz_path = f"{checkpoint_dir}/{last_filename}"

def on_fit_epoch_end(trainer):

    run_dir = Path("/content/") / trainer.save_dir.resolve()
    print("start archiving...")
    !tar -czvf {last_filename} -C {run_dir} . > /dev/null
    print("end archiving...")

    if trainer.epoch > 0 and os.path.exists(last_gz_path):
      print("Rename prev file")
      !mv "{last_gz_path}" "{checkpoint_dir}/epoch/epoch{trainer.epoch + 1}.tar.gz"

    !cp {last_filename} "{checkpoint_dir}"
    !rm {last_filename}
    print("backup complete!")


is_resume = False
model = "/content/runs/detect/scratch-detector/weights/last.pt"

if os.path.exists(model):
  is_resume = True
  print("Resume training")
else:
  model = 'yolo11m.pt'
  print("Start training")

model = YOLO(model)
model.add_callback('on_fit_epoch_end', on_fit_epoch_end)

results = model.train(
    data='/content/merged/data.yaml',
    epochs=200,
    imgsz=640,
    batch=12,
    name="scratch-detector",
    device=0,
    val=True,
    save=True,
    cache=False,
    save_dir="/content/runs/",
    resume=is_resume,
    lr0=0.001,
    lrf=0.05,
    cos_lr=True,
    warmup_epochs=4,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    patience=15,
    mosaic=0.0,
    mixup=0.0,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    fliplr=0.1,
    flipud=0.0,
    translate=0.05,
    scale=0.05,
    perspective=0.0,
    freeze=0
)

/content
Resume training
New https://pypi.org/project/ultralytics/8.3.156 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/merged/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/scratch-detector/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=F

KeyboardInterrupt: 

In [ ]:
#!rm -rf /content/content
# !rm -rf /content/merged/
!rm -rf /content/runs
#!rm -rf /content/gdrive/MyDrive/models/checkpoints/test/
#!mkdir -P /content/gdrive/MyDrive/models/checkpoints/test/
#!ls /content/gdrive/MyDrive/models/checkpoints/test/
